In [130]:
import pyzx as zx

In [131]:
from pyzx import VertexType, Graph, sqasm, draw
from sympy import Symbol, pi

s = """
qreg Q[1];
qreg q[4];

cx Q[0], q[0];
cx Q[0], q[1];  
cx Q[0], q[2];
cx Q[0], q[3];
"""
m = Symbol('m', boolean=True)
g = sqasm(s)
g.set_phase(6, m*pi)
gd = g.copy().adjoint() #daggar
gc = g.tensor(gd) #channel

#extract classical bit
gc.add_vertex(VertexType.X, -1, 6)
gc.add_edge((5,26))
gc.add_edge((18,26))
gc.normalize()


In [132]:
zx.draw(g, True)
zx.draw(gd, True)
zx.draw(gc, True)

In [133]:
corr = """
qreg q[8];
x q[0];
x q[4];
"""
correction = sqasm(corr)
gtot =  correction * gc  
gtot.add_edge((26,35))
gtot.add_edge((26,36))
zx.full_reduce(gtot)
draw(gtot, True)

In [134]:
import pyzx as zx
import numpy as np
import quimb as qu
import quimb.tensor as qtn

def trace(g, qubits):
    gc = g.copy()
    inp_verts = gc.inputs()
    out_verts = gc.outputs()
    for q in qubits:
        v1 = inp_verts[q]
        v2 = out_verts[q]
        print(f"Adding edge between {v1} and {v2}")
        gc.set_type(v1, zx.VertexType.X)
        gc.set_type(v2, zx.VertexType.X)
        gc.add_edge((v1, v2))
    return gc

g = zx.generate.cnots(7,20)
g.apply_state('0///000')
g.apply_effect('//////0')
zx.draw(g, True)
rho = g.tensor(g.copy().adjoint())
zx.draw(rho, True)
gt = trace(rho, [-1, -2, -3])
zx.draw(gt, True)

Adding edge between 130 and 71
Adding edge between 131 and 70
Adding edge between 132 and 69
